In [1]:
import logging
import math
import wandb
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

import pandas as pd
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
SAVE_EVERY_N_EPOCHS = 4

model_args = Seq2SeqArgs()
model_args.num_train_epochs = 2
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = False
model_args.tensorboard_dir = "runs"
model_args.max_length = 50
model_args.train_batch_size=10
model_args.overwrite_output_dir=True
model_args.wandb_project = "cs224u"
model_args.save_steps = steps_per_epoch = math.floor(6000 / SAVE_EVERY_N_EPOCHS) * SAVE_EVERY_N_EPOCHS
model_args.save_model_every_epoch = False

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=True,
)

# model = Seq2SeqModel(
#     encoder_type="bert",
#     encoder_name="bert-base-uncased",
#     decoder_name="bert-base-uncased",
#     args=model_args,
#     use_cuda=True,
# )


def count_matches(labels, preds):
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )

In [2]:
import pandas as pd

train_df = pd.read_pickle("train.pkl")
train_df = train_df.dropna()
dev_df = pd.read_pickle("dev.pkl")

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [3]:
# train_df = train_df.head(1000)

In [4]:
# dev_df = dev_df.head(20)

In [5]:
# Train the model
wandb.init(project='cs224u', entity='gbanerje')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

# Model training here


model.train_model(
    train_df, eval_data=dev_df, matches=count_matches, show_running_loss=True, args={'fp16': False}
)

wandb.join()

# # Evaluate the model
results = model.eval_model(dev_df)

# # Use the model for prediction
# print(
#     model.predict(
#         [
#             "she beowright"
#         ]
#     )
# )

wandb: Currently logged in as: gbanerje (use `wandb login --relogin` to force relogin)


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model: Training started


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-100-epoch-1
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-200-epoch-2
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


NameError: name 'loss' is not defined

In [6]:
wandb.join()

# # Evaluate the model
results = model.eval_model(dev_df)

Training loss,2.35594
lr,0.0
global_step,200
_runtime,193
_timestamp,1622363757
_step,5
eval_loss,2.27646
train_loss,2.35594
matches,0


Training loss,▂█▂▁
lr,█▆▃▁
global_step,▁▃▃▆██
_runtime,▁▃▃▆██
_timestamp,▁▃▃▆██
_step,▁▂▄▅▇█
eval_loss,█▁
train_loss,█▁
matches,▁▁


INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 2.276461362838745, 'matches': 0}


In [23]:
model.predict(
        [
            "Hee walks dogks"
        ]
    )

['Hee walks dog walks.']

In [9]:
model.predict(
        [
            "Hee woks dogks"
        ]
    )

['Hee woks!']

In [6]:
model